In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.0.8'

# 5.1 - Введение в коннеты

Для начала давайте взглянем на очень простой пример с коннетом. Мы будем использовать коннет для классификации цифр MNIST, задача, которую вы уже выполняли в главе 2, используя плотно подключенную сеть (точность нашего теста составляла 97,8%). Хотя наш коннет будет очень простым, его точность все еще будет зависеть от точности модели с плотной связью из главы 2.

6 строк кода ниже показывают, как выглядит базовый коннет. Это стек слоев `Conv2D` и` MaxPooling2D`. Посмотрим, что они делают конкретно.
Важно, что коннет принимает в качестве входных тензоров форму `(image_height, image_width, image_channels)` (не включая размер пакета).
В нашем случае мы сконфигурируем наш convnet для обработки входных данных размера `(28, 28, 1)`, который является форматом изображений MNIST. Мы делаем это через передачу аргумента `input_shape = (28, 28, 1)` нашему первому слою.

In [2]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

Взглянем на архитектуру коннета:

In [3]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


Вы можете увидеть, что выходные данные каждого слоя `Conv2D` и` MaxPooling2D` являются трехмерным тензором формы `(высота, ширина, каналы)`. Ширина и размеры имеют тенденцию уменьшаться, когда мы углубляемся в сеть. Количество каналов контролируется первым аргументом, переданным
слоям `Conv2D` (например, 32 или 64).

Следующим шагом будет подача нашего последнего выходного тензора (формы `(3, 3, 64)`) в плотно связанную сеть классификаторов, с такой вы уже знакомы: стеком "плотных" слоев. Эти классификаторы обрабатывают  одномерные векторы, в то время как наш текущий вывод является трехмерным тензором.
Итак, сначала нам нужно сгладить наши 3D-выходы в 1D, а затем добавить несколько «плотных» слоев сверху:

In [4]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

Мы сделаем классификацию с 10 путями, поэтому мы будем использовать последний слой с 10 выходами и активацией softmax. Теперь наша сеть выглядит как:

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                36928     
__________

Как видите, наши выходы `(3, 3, 64)` были сплющены в векторы формы `(576,)`прежде чем проходить через два "плотных" слоя.

Теперь давайте обучим наш коннет цифрам MNIST. Мы будем использовать довольно много кода, уже рассмотренного в примере MNIST из главы
2.

In [6]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [7]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
60000/60000 [==============================] - 8s - loss: 0.1766 - acc: 0.9440     
Epoch 2/5
60000/60000 [==============================] - 7s - loss: 0.0462 - acc: 0.9855     
Epoch 3/5
60000/60000 [==============================] - 7s - loss: 0.0322 - acc: 0.9902     
Epoch 4/5
60000/60000 [==============================] - 7s - loss: 0.0241 - acc: 0.9926     
Epoch 5/5
60000/60000 [==============================] - 7s - loss: 0.0187 - acc: 0.9943     


Давайте проверим модель на тестовых данных:

In [8]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

 9536/10000 [===========================>..] - ETA: 0s

In [9]:
test_acc

0.99129999999999996

В то время как наша плотно связанная сеть из Главы 2 имела точность теста 97,8%, наша базовая сеть Connet имеет точность теста 99,3%: мы снизили уровень ошибок на 68% (относительно). Not bad! 